In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [46]:
train_data_orig = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data = train_data_orig
train_data.head()

In [47]:
test_data_orig = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data = test_data_orig
test_data.head()

In [ ]:
plt.plot(train_data['Fare'], '.', color='b')
plt.plot(train_data[train_data['Pclass']==1]['Fare'], '.', color='r')

In [ ]:
train_data.shape, test_data.shape, train_data.dtypes

In [ ]:
plt.plot(train_data['Fare'], '.', color='b')
plt.plot(train_data[train_data['Sex']=='male']['Fare'], '.', color='r')

In [ ]:
plt.plot(train_data['Fare'], '.', color='b')
plt.plot(train_data[train_data['Survived']==1]['Fare'], '.', color='r')

In [31]:
train_data[train_data['Pclass']==3]

In [ ]:
train_data[train_data['Cabin'].isna()==False]

In [4]:
# import library
import seaborn as sns
import matplotlib.pyplot as plt

import sklearn


from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier

from sklearn.linear_model import LogisticRegression

from sklearn.metrics import f1_score, confusion_matrix, classification_report
from sklearn.model_selection import learning_curve, train_test_split
from sklearn.model_selection import learning_curve, GridSearchCV



In [5]:
def evaluation(model, X_train, y_train, X_test, y_test, scoring, plot):

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    print("test set : ")
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred)) 
    print("train set : ")
    print(confusion_matrix(y_train, model.predict(X_train)))  
    print(classification_report(y_train, model.predict(X_train) ))

    if plot == True:
        N, train_score, val_score = learning_curve(model, X_train, y_train, shuffle=True, cv = 4, scoring = scoring, train_sizes = np.linspace(0.1, 1, 6) )
        N_A, train_score_A, val_score_A = learning_curve(model, X_train, y_train, shuffle=True, cv = 4, scoring = 'accuracy', train_sizes = np.linspace(0.1, 1, 6) )

        plt.figure(figsize = (12,6))
        plt.subplot(121)
        plt.title(scoring)
        plt.plot(N, train_score.mean(axis=1), label ="train_score")
        plt.plot(N, val_score.mean(axis=1), label ="cross_validation_score")
        plt.legend()
        plt.subplot(122)
        plt.title('accuracy')
        plt.plot(N_A, train_score_A.mean(axis=1), label ="train_score")
        plt.plot(N_A, val_score_A.mean(axis=1), label ="cross_validation_score")
        plt.legend()
        
        
        
    else :
        print("GRAPHIQUE NON DEMANDE")
    #return y_pred

In [ ]:
############################################################ DATA EXPLORATORY ############################################################
##########################################################################################################################################

for i in ['Survived','Sex', 'Pclass','Age']:
    
    if i != 'Age':
        plt.figure(figsize = (8,5))
    else:
        plt.figure(figsize = (20,5))
    
    plt.title(i)
    (train_data[i].value_counts(dropna = False, normalize=True).plot.bar())
    



In [48]:
####################################### pre-processing #############################################

label_encoder = LabelEncoder()
label_encoder.fit(['male', 'female'])
train_data['Sex'] = label_encoder.transform(train_data['Sex'])
test_data['Sex'] = label_encoder.transform(test_data['Sex'])

####################################################################################################

In [ ]:
train_data['Sex'].value_counts()


In [ ]:
 
######################################################## FIRST MODEL ################################################################

train_set = train_data[['PassengerId','Pclass','Sex','Survived']]
test_set = test_data[['PassengerId','Pclass','Sex']]

nb_var = train_set.shape[1]-1

train_X_s = (train_set.iloc[:,1:nb_var]).to_numpy()
train_Y_s = np.asanyarray(train_set.iloc[:,nb_var])

test_X_s = (test_set.iloc[:,1:nb_var]).to_numpy()
#test_Y_s = np.asanyarray(test_set.iloc[:,nb_var])


In [ ]:
model_DecisionTreeClassifier = DecisionTreeClassifier(random_state = 0, class_weight = "balanced" )
param = { 'criterion': ["gini", "entropy"],
         'min_samples_split' : [5, 10, 20, 40, 50, 80, 100] }

grid_DecisionTreeClassifier = GridSearchCV(model_DecisionTreeClassifier, param_grid= param, cv = 4, scoring = "recall", n_jobs=-1)
resultat_grid_DecisionTreeClassifier = grid_DecisionTreeClassifier.fit(train_X_s, train_Y_s)
resultat_grid_DecisionTreeClassifier.best_params_

In [ ]:
final_model_DecisionTreeClassifier = DecisionTreeClassifier(random_state = 0, class_weight = "balanced", criterion ='gini', min_samples_split=5 )
evaluation(final_model_DecisionTreeClassifier, train_X_s, train_Y_s, test_X_s, None, scoring="recall", plot=True)


In [ ]:
model_RandomForestClassifier = RandomForestClassifier(class_weight="balanced", random_state =0 )
param = {'criterion': ['gini','entropy'] ,'n_estimators' : [3,5,10,20,50,100,200], 'min_samples_split' : [5, 10, 20, 40, 50, 80, 100] }

grid_RandomForestClassifier = GridSearchCV(model_RandomForestClassifier, param, cv = 4, scoring = "recall", n_jobs=-1)
resultat_grid_RandomForestClassifier = grid_RandomForestClassifier.fit(train_X_s, train_Y_s)
resultat_grid_RandomForestClassifier.best_params_

In [ ]:
final_model_RandomForestClassifier = RandomForestClassifier(class_weight="balanced", criterion="gini", n_estimators = 20, random_state=0, min_samples_split = 100) 
evaluation(final_model_RandomForestClassifier, train_X_s, train_Y_s, test_X_s, None, scoring="recall", plot=True)

In [ ]:
####################################################
####################################################

In [ ]:
######################################################## ADD A COLUMN IN DATASET ################################################################
train_set = train_data[['PassengerId','Pclass','Sex', 'Age','Survived']]
test_set = test_data[['PassengerId','Pclass','Sex', 'Age']]


In [ ]:
train_set[train_set['Age'].isna()]

In [ ]:
for i in ['Pclass', 'Sex', 'Survived']: 
    print(train_set[train_set['Age'].isna()][i].value_counts()); print('\n')

In [ ]:
##############################    ANALYSIS NAN    #############################

plt.figure(figsize=(10,5))
sns.countplot( x="Sex" , hue="Survived" , data=train_set[train_set['Age'].isna()])

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot( x="Pclass" , hue="Survived" , data=train_set[train_set['Age'].isna()])

plt.figure(figsize=(10,5))
sns.catplot( x="Pclass" , hue="Survived", col='Sex' , data=train_set[train_set['Age'].isna()], kind="count")

In [ ]:
plt.plot(train_set['Age'].value_counts(dropna=True), '.')

In [ ]:
plt.figure(figsize=(20,5))
plt.title('AGE')
train_data['Age'].value_counts(dropna = True, normalize=True).sort_index().plot.bar()

plt.figure(figsize=(20,5))
plt.title('Sex')
plt.hist(train_data['Age'],bins =40)
plt.hist(train_data[train_data['Sex']==1]['Age'],bins = 40)

plt.figure(figsize=(20,5))
plt.title('Suvived')
plt.hist(train_data['Age'],bins = 40)
plt.hist(train_data[train_data['Survived']==1]['Age'],bins = 40)

In [ ]:
train_set['Age']

In [ ]:

    ######################################################## SECOND MODEL ################################################################


In [ ]:
def change_age(x):
    x.fillna('25', inplace =True)
    x['Age']=x['Age'].astype(float)
    x.loc[x['Age']<19,'Age'] = 0
    x.loc[x['Age']>=19,'Age'] = 1

    return x

In [ ]:
train_set = train_data[['PassengerId','Pclass','Sex', 'Age','Survived']]
test_set = test_data[['PassengerId','Pclass','Sex', 'Age']]


change_age(train_set)
change_age(test_set)


nb_var = train_set.shape[1]-1

train_X_s = (train_set.iloc[:,1:nb_var]).to_numpy()
train_Y_s = np.asanyarray(train_set.iloc[:,nb_var])

test_X_s = (test_set.iloc[:,1:nb_var]).to_numpy()
#test_Y_s = np.asanyarray(test_set.iloc[:,nb_var])


In [ ]:
model_DecisionTreeClassifier = DecisionTreeClassifier(random_state = 0 , class_weight = "balanced")
param = { 'criterion': ["gini", "entropy"], 'max_features':[2,3],
         'min_samples_split' : [5, 10, 20, 40, 50, 80 ,100] }

grid_DecisionTreeClassifier = GridSearchCV(model_DecisionTreeClassifier, param_grid= param, cv = 4, scoring = "recall", n_jobs=-1)
resultat_grid_DecisionTreeClassifier = grid_DecisionTreeClassifier.fit(train_X_s, train_Y_s)
resultat_grid_DecisionTreeClassifier.best_params_

In [ ]:
final_model_DecisionTreeClassifier = DecisionTreeClassifier(random_state = 0, class_weight = "balanced", criterion ='gini', min_samples_split=5, max_features = 2 )

evaluation(final_model_DecisionTreeClassifier, train_X_s, train_Y_s, test_X_s, None, scoring="recall", plot=True)


In [ ]:
model_RandomForestClassifier = RandomForestClassifier(class_weight="balanced", random_state =0 )
param = {'criterion': ['gini','entropy'] ,'n_estimators' : [3,5,10,20,50,100,200], 'max_features':[2,3], 'min_samples_split' : [5, 10, 20, 40, 50, 80,100] }

grid_RandomForestClassifier = GridSearchCV(model_RandomForestClassifier, param, cv = 4, scoring = "recall", n_jobs=-1)
resultat_grid_RandomForestClassifier = grid_RandomForestClassifier.fit(train_X_s, train_Y_s)
resultat_grid_RandomForestClassifier.best_params_

In [ ]:
final_model_RandomForestClassifier = RandomForestClassifier(class_weight="balanced", criterion="gini", n_estimators = 10, random_state=0, max_features=2, min_samples_split=5) 
evaluation(final_model_RandomForestClassifier, train_X_s, train_Y_s, test_X_s, None, scoring="recall", plot=True)

In [ ]:
pd.set_option("display.max_rows", 50)
train_data[train_data['Age']<10]

In [ ]:
# La pluplart des hommes sont morts dans S
plt.figure(figsize=(10,5))
sns.countplot( x="Embarked" , hue="Survived" , data=train_data)

plt.figure(figsize=(10,5))
sns.catplot( x="Embarked" , hue="Survived", col='Sex' , data=train_data, kind="count")

In [ ]:
plt.figure(figsize=(25,5))
sns.countplot( x="Cabin" , hue="Survived" , data=train_data)

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot( x="Sex" , hue="Survived" , data=train_set)

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot( x="Pclass" , hue="Survived" , data=train_set)

plt.figure(figsize=(10,5))
sns.catplot( x="Pclass" , hue="Survived", col='Sex' , data=train_set, kind='count')

In [ ]:
def change_age_v2(x):
    x.loc[:,'Age'].fillna('200', inplace =True)
    x.loc[:,'Age']=x.loc[:,'Age'].astype(float)
    
    x.loc[(x['Sex']==0)&(x['Age']==200) , 'Age'] = 18
    
    
    x.loc[x['Age']<8,'Age'] = 0
    x.loc[(x['Age']>=8)&(x['Age']<19),'Age'] = 1
    x.loc[(x['Age']>=19)&(x['Age']<33),'Age'] = 2
    x.loc[(x['Age']>=33)&(x['Age']<48),'Age'] = 3
    x.loc[(x['Age']>=48)&(x['Age']<65),'Age'] = 4
    x.loc[x['Age']>=65,'Age'] = 5

    return x

In [ ]:
train_set = train_data[['PassengerId','Pclass','Sex', 'Age','Survived']]
test_set = test_data[['PassengerId','Pclass','Sex', 'Age']]


change_age_v2(train_set)
change_age_v2(test_set)


nb_var = train_set.shape[1]-1

train_X_s = (train_set.iloc[:,1:nb_var]).to_numpy()
train_Y_s = np.asanyarray(train_set.iloc[:,nb_var])

test_X_s = (test_set.iloc[:,1:nb_var]).to_numpy()
#test_Y_s = np.asanyarray(test_set.iloc[:,nb_var])


In [ ]:
train_set['Age'].value_counts()

In [ ]:
model_DecisionTreeClassifier = DecisionTreeClassifier(random_state = 0 , class_weight = "balanced")
param = { 'criterion': ["gini", "entropy"], 'max_features':[2,3],
         'min_samples_split' : [5, 10, 20, 40, 50, 100] }

grid_DecisionTreeClassifier = GridSearchCV(model_DecisionTreeClassifier, param_grid= param, cv = 4, scoring = "recall", n_jobs=-1)
resultat_grid_DecisionTreeClassifier = grid_DecisionTreeClassifier.fit(train_X_s, train_Y_s)
resultat_grid_DecisionTreeClassifier.best_params_

In [ ]:
final_model_DecisionTreeClassifier = DecisionTreeClassifier(random_state = 0, class_weight = "balanced", criterion ='gini', min_samples_split=100, max_features = 3 )

evaluation(final_model_DecisionTreeClassifier, train_X_s, train_Y_s, test_X_s, None, scoring="recall", plot=True)


In [ ]:
model_RandomForestClassifier = RandomForestClassifier(class_weight="balanced", random_state =0 )
param = {'criterion': ['gini','entropy'] ,'n_estimators' : [3,5,10,20,50,100,200], 'max_features':[2,3], 'min_samples_split' : [5, 10, 20, 40, 50,80, 100] }

grid_RandomForestClassifier = GridSearchCV(model_RandomForestClassifier, param, cv = 4, scoring = "recall", n_jobs=-1)
resultat_grid_RandomForestClassifier = grid_RandomForestClassifier.fit(train_X_s, train_Y_s)
resultat_grid_RandomForestClassifier.best_params_

In [ ]:
final_model_RandomForestClassifier = RandomForestClassifier(random_state=0, class_weight="balanced", criterion="gini", n_estimators = 10, max_features=2, min_samples_split=80) 
evaluation(final_model_RandomForestClassifier, train_X_s, train_Y_s, test_X_s, None, scoring="recall", plot=True)

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot( x="Age" , hue="Survived" , data=train_set)

plt.figure(figsize=(10,5))
sns.catplot( x="Age" , hue="Survived", col='Sex' , data=train_set, kind='count')

In [ ]:

    ######################################################## THIRD MODEL ################################################################


In [ ]:
def change_age_v3(x):
    x.loc[:,'Age'].fillna('200', inplace =True); print('OK1')
    x.loc[:,'Age']=x.loc[:,'Age'].astype(float); print('OK2')
    
    x.loc[(x['Sex']==0)&(x['Pclass']==1)&(x['Age']==200), 'Age'] = 15; print('OK3')
    x.loc[(x['Sex']==0)&(x['Pclass']==2)&(x['Age']==200), 'Age'] = 15; print('OK3')
    x.loc[(x['Sex']==0)&(x['Pclass']==3)&(x['Age']==200), 'Age'] = 25; print('OK3')
    
    x.loc[(x['Sex']==1)&(x['Pclass']==1)&(x['Age']==200), 'Age'] = 45; print('OK3')
    x.loc[(x['Sex']==1)&(x['Pclass']==2)&(x['Age']==200), 'Age'] = 45; print('OK3')
    x.loc[(x['Sex']==1)&(x['Pclass']==3)&(x['Age']==200), 'Age'] = 65; print('OK3')
    

    x.loc[x['Age']<12,'Age'] = 0; print('OK4')
    x.loc[(x['Age']>=12)&(x['Age']<19),'Age'] = 1; print('OK5')
    x.loc[(x['Age']>=19)&(x['Age']<33),'Age'] = 1; print('OK6')
    x.loc[(x['Age']>=33)&(x['Age']<40),'Age'] = 1; print('OK7')
    x.loc[(x['Age']>=40)&(x['Age']<65),'Age'] = 2; print('OK8')
    x.loc[(x['Age']>=65)&(x['Age']<77),'Age'] = 3; print('OK9')
    x.loc[x['Age']>=77,'Age'] = 0; print('OK10')
    

def change_Pclass(x):
    x.loc[(x['Pclass']==1)|(x['Pclass']==2),'Pclass']=0
    x.loc[(x['Pclass']==3),'Pclass']=1

def change_SibSp(x):
    x.loc[x['SibSp']!=0,'SibSp'] = 1
    
def change_Parch(x):
    x.loc[x['Parch']!=0,'Parch'] = 1

def change_Embarked(x):
    x.loc[:,'Embarked'].fillna('C', inplace=True)
    x.loc[x['Embarked']=='S','Embarked']='0'
    x.loc[x['Embarked']=='C','Embarked']='1'
    x.loc[x['Embarked']=='Q','Embarked']='1'
    
def change_Fare(x):
    
    x.loc[:,'Fare'].fillna(31, inplace=True)
    x.loc[x['Fare']<7.91, 'Fare'] = 0
    x.loc[(x['Fare']>=7.91)&(x['Fare']<14.45), 'Fare'] = 1
    x.loc[(x['Fare']>=14.45)&(x['Fare']<31), 'Fare'] = 2
    x.loc[x['Fare']>=31, 'Fare'] = 3


In [ ]:
#train_set = train_data[['PassengerId','Pclass','Sex', 'Age','SibSp','Parch','Embarked','Survived']]
#test_set = test_data[['PassengerId','Pclass','Sex', 'Age','SibSp','Parch','Embarked']]

train_set = train_data[['PassengerId','Pclass','Sex', 'Age','Parch','Embarked', 'Fare', 'Survived']]
test_set = test_data[['PassengerId','Pclass','Sex', 'Age','Parch','Embarked', 'Fare']]


change_age_v3(train_set)
change_age_v3(test_set)

change_Pclass(train_set)
change_Pclass(test_set)

#change_SibSp(train_set)
#change_SibSp(test_set)

change_Parch(train_set)
change_Parch(test_set)

change_Embarked(train_set)
change_Embarked(test_set)


change_Fare(train_set)
change_Fare(test_set)


nb_var = train_set.shape[1]-1

train_X_s = (train_set.iloc[:,1:nb_var]).to_numpy()
train_Y_s = np.asanyarray(train_set.iloc[:,nb_var])

test_X_s = (test_set.iloc[:,1:nb_var]).to_numpy()
#test_Y_s = np.asanyarray(test_set.iloc[:,nb_var])

In [ ]:
train_set['Pclass'].value_counts(), train_set['Parch'].value_counts(), train_set['Age'].value_counts(dropna=False), train_set['Embarked'].value_counts(dropna=False),train_set['Sex'].value_counts()

In [ ]:
model_DecisionTreeClassifier = DecisionTreeClassifier(random_state = 0 , class_weight = "balanced")
param = { 'criterion': ["gini", "entropy"], 'max_features':[2,3,4,5,6],
         'min_samples_split' : [5, 10, 20, 40, 50, 100] }

grid_DecisionTreeClassifier = GridSearchCV(model_DecisionTreeClassifier, param_grid= param, cv = 4, scoring = "accuracy", n_jobs=-1)
resultat_grid_DecisionTreeClassifier = grid_DecisionTreeClassifier.fit(train_X_s, train_Y_s)
resultat_grid_DecisionTreeClassifier.best_params_

In [ ]:
final_model_DecisionTreeClassifier = DecisionTreeClassifier(random_state = 0, class_weight = "balanced", criterion ='gini', min_samples_split=100, max_features = 3 )

evaluation(final_model_DecisionTreeClassifier, train_X_s, train_Y_s, test_X_s, None, scoring="recall", plot=True)


In [ ]:
model_RandomForestClassifier = RandomForestClassifier(class_weight="balanced", random_state =0 )
param = {'criterion': ['gini','entropy'] ,'n_estimators' : [3,5,10,20,50,100], 'max_features':[2,3,4,5,6], 'min_samples_split' : [5, 10, 20, 40, 50,80, 100] }

grid_RandomForestClassifier = GridSearchCV(model_RandomForestClassifier, param, cv = 4, scoring = "accuracy", n_jobs=-1)
resultat_grid_RandomForestClassifier = grid_RandomForestClassifier.fit(train_X_s, train_Y_s)
resultat_grid_RandomForestClassifier.best_params_

In [ ]:
final_model_RandomForestClassifier = RandomForestClassifier(random_state=0, class_weight="balanced", criterion="gini", n_estimators = 20, max_features=3, min_samples_split=40) 
y_pred = evaluation(final_model_RandomForestClassifier, train_X_s, train_Y_s, test_X_s, None, scoring="recall", plot=True)

In [ ]:
#On conserve les noms de variable à part
liste_variables = list(train_set.columns[1:nb_var])
importances = final_model_RandomForestClassifier.feature_importances_
indices = np.argsort(importances)
# style du graphique 
plt.style.use('fivethirtyeight')
%matplotlib inline
plt.figure(1)
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [liste_variables[i] for i in indices])
plt.xlabel('Relative Importance')

In [ ]:
model_AdaBoost = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(class_weight = "balanced"), random_state =0 )
param = {'base_estimator__criterion': ['gini','entropy'], 'base_estimator__min_samples_split' : [5, 10, 20, 40, 50,80, 100],
         'base_estimator__max_features':[2,3,4,5,6] , 'n_estimators' : [3,5,10,20,50,80],'learning_rate':[0.001,0.01,0.1,1] }

grid_AdaBoost = GridSearchCV(model_AdaBoost, param, cv = 4, scoring = "recall", n_jobs=-1)
resultat_grid_AdaBoost = grid_AdaBoost.fit(train_X_s, train_Y_s)
resultat_grid_AdaBoost.best_params_

In [ ]:
final_model_AdaBoost = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(class_weight="balanced", criterion="gini", max_features=3, min_samples_split=80),n_estimators = 3, learning_rate=0.001, random_state=0 ) 
evaluation(final_model_AdaBoost, train_X_s, train_Y_s, test_X_s, None, scoring="recall" , plot =True)

In [ ]:
model_RandomForestClassifier = RandomForestClassifier(random_state =0 )
param = {'criterion': ['gini','entropy'] ,'n_estimators' : [3,5,10,20,50,100], 'max_features':[2,3,4,5,6], 'min_samples_split' : [5, 10, 20, 40, 50,80, 100] }

grid_RandomForestClassifier = GridSearchCV(model_RandomForestClassifier, param, cv = 4, scoring = "accuracy", n_jobs=-1)
resultat_grid_RandomForestClassifier = grid_RandomForestClassifier.fit(train_X_s, train_Y_s)
resultat_grid_RandomForestClassifier.best_params_

In [ ]:
final_model_RandomForestClassifier = RandomForestClassifier(random_state=0, criterion="entropy", n_estimators = 20, max_features=2, min_samples_split=20) 
y_pred = evaluation(final_model_RandomForestClassifier, train_X_s, train_Y_s, test_X_s, None, scoring="recall", plot=True)

In [ ]:
model_AdaBoost = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(), random_state =0 )
param = {'base_estimator__criterion': ['gini','entropy'], 'base_estimator__min_samples_split' : [5, 10, 20, 40, 50,80, 100],
         'base_estimator__max_features':[2,3,4,5,6] , 'n_estimators' : [3,5,10,20,50], 'learning_rate':[0.001, 0.01,0.1,1] }

grid_AdaBoost = GridSearchCV(model_AdaBoost, param, cv = 4, scoring = "accuracy", n_jobs=-1)
resultat_grid_AdaBoost = grid_AdaBoost.fit(train_X_s, train_Y_s)
resultat_grid_AdaBoost.best_params_

In [ ]:
final_model_AdaBoost = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(criterion="gini", max_features=3, min_samples_split=20),n_estimators = 5, learning_rate=0.001, random_state=0 ) 
evaluation(final_model_AdaBoost, train_X_s, train_Y_s, test_X_s, None, scoring="recall" , plot =True)

In [49]:
def change_name(train_data):
    train_data['name_categorie']=9
    for i in range(train_data.shape[0]):
        if (train_data.loc[i,'Name'].find('Mr.')!=-1)|(train_data.loc[i,'Name'].find('Rev.')!=-1)|(train_data.loc[i,'Name'].find('Dr.')!=-1)|(train_data.loc[i,'Name'].find('Don.')!=-1)|(train_data.loc[i,'Name'].find('Major.')!=-1)|(train_data.loc[i,'Name'].find('Col.')!=-1)|(train_data.loc[i,'Name'].find('Sir.')!=-1)|(train_data.loc[i,'Name'].find('Capt.')!=-1):
            train_data.loc[i, 'name_categorie'] = 4
        elif (train_data.loc[i,'Name'].find('Mrs')!=-1)|(train_data.loc[i,'Name'].find('Mme.')!=-1)|(train_data.loc[i,'Name'].find('Ms.')!=-1)|(train_data.loc[i,'Name'].find('Dona.')!=-1):
            train_data.loc[i, 'name_categorie'] = 3 
        elif (train_data.loc[i,'Name'].find('Miss')!=-1)|(train_data.loc[i,'Name'].find('Mlle.')!=-1)|(train_data.loc[i,'Name'].find('Countess.')!=-1):
            train_data.loc[i, 'name_categorie'] = 2
        elif train_data.loc[i,'Name'].find('Master')!=-1:
            train_data.loc[i, 'name_categorie'] = 1
        else:
            train_data.loc[i, 'name_categorie'] = 1 # Mr.


def change_age_v3(x):
    #x.loc[:,'Age'].fillna('200', inplace =True); print('OK1')
    x.loc[:,'Age']=x.loc[:,'Age'].astype(float); print('OK2')    

    x.loc[x['Age']<15,'Age'] = 0; print('OK4')
    #x.loc[(x['Age']>=15)&(x['Age']<18),'Age'] = 1; print('OK5')
    x.loc[(x['Age']>=15)&(x['Age']<=30),'Age'] = 1; print('OK5')
    x.loc[(x['Age']>30)&(x['Age']<=45),'Age'] = 2; print('OK6')
    #x.loc[(x['Age']>45)&(x['Age']<=55),'Age'] = 4; print('OK7')
    #x.loc[(x['Age']>45)&(x['Age']<=55),'Age'] = 5; print('OK8')
    x.loc[x['Age']>45,'Age'] = 3
    

def change_Pclass(x):
    x.loc[(x['Pclass']==1)|(x['Pclass']==2),'Pclass']=0
    x.loc[(x['Pclass']==3),'Pclass']=1

def change_SibSp(x):
    x.loc[x['SibSp']>2,'SibSp'] = 2
    
    
def change_Parch(x):
    x.loc[x['Parch']>2,'Parch'] = 2

def change_Embarked(x):
    x.loc[:,'Embarked'].fillna('C', inplace=True)
    x.loc[x['Embarked']=='S','Embarked']='0'
    x.loc[x['Embarked']=='C','Embarked']='1'
    x.loc[x['Embarked']=='Q','Embarked']='2'
    x.loc[:,'Embarked']=x.loc[:,'Embarked'].astype(int); 
    
def change_Fare(x):
    x.loc[:,'Fare'].fillna(31, inplace=True)
    x.loc[x['Fare']<7.91, 'Fare'] = 0
    x.loc[(x['Fare']>=7.91)&(x['Fare']<14.45), 'Fare'] = 1
    x.loc[(x['Fare']>=14.45)&(x['Fare']<31), 'Fare'] = 2
    x.loc[x['Fare']>=31, 'Fare'] = 3

def Ticket_categorie(x):
    test_test = pd.DataFrame({ 'Ticket' : x['Ticket'].value_counts().index, 'Ticket_categorie':x['Ticket'].value_counts()[:] } )
    test_test.loc[test_test['Ticket_categorie']>=3,'Ticket_categorie']=3
    test_test = x.merge(test_test, on = 'Ticket', how='inner' )
    
    return test_test    
    

def estimator_null(x):
    x['y_pred'] = 9999 ; print('ok')
    #x.loc[(x['Sex']==1)&((x['SibSp']==0)|(x['SibSp']==2))&(x['Parch']==0), 'y_pred']=1; # &(x['Embarked']=='2')
    x.loc[(x['Sex']==1)&((x['SibSp']==0)|(x['SibSp']==2))&(x['Parch']==0), 'y_pred']=0 #  &(x['Embarked']!='2')
    #x.loc[(x['Sex']==1)&(x['SibSp']==0)&(x['Parch']==0)&(x['Embarked']=='0'), 'y_pred']=1 #---
    x.loc[(x['Sex']==1)&(x['SibSp']==1)&(x['Parch']==0), 'y_pred']=1
    x.loc[(x['Sex']==1)&(x['SibSp']==0)&(x['Parch']==1), 'y_pred']=1 ; print('ok')
   
    x.loc[(x['Sex']==1)&(x['SibSp']>2), 'y_pred']=0

    x.loc[(x['Sex']==1)&(x['SibSp']==1)&(x['Parch']==1), 'y_pred']=1
    x.loc[(x['Sex']==1)&(x['SibSp']==1)&(x['Parch']>=2), 'y_pred']=0
    
    
    x.loc[(x['Sex']==0)&((x['SibSp']==0)|(x['SibSp']==1))&((x['Parch']==1)|(x['Parch']==0)), 'y_pred']=1
    #x.loc[(x['Sex']==0)&(x['SibSp']==0)&(x['Parch']==0)&(x['Embarked']=='0'), 'y_pred']=0 #---
    x.loc[(x['Sex']==0)&(x['SibSp']==2), 'y_pred']=1
    x.loc[(x['Sex']==0)&(x['SibSp']>2), 'y_pred']=0
    x.loc[(x['Sex']==0)&((x['SibSp']==0)|(x['SibSp']==1))&(x['Parch']>=2), 'y_pred']=0
    
    return x
    ######################################################## FOURTH MODEL ################################################################

#train_set = train_data[['PassengerId','Pclass','Sex', 'Age','SibSp','Parch','Embarked','Survived']]
#test_set = test_data[['PassengerId','Pclass','Sex', 'Age','SibSp','Parch','Embarked']]

change_name(train_data)
change_name(test_data)

train_data = Ticket_categorie(train_data)
test_data = Ticket_categorie(test_data)

train_set = train_data[['PassengerId','name_categorie','Pclass','Sex', 'Age','SibSp','Parch','Embarked', 'Ticket_categorie', 'Survived']]
test_set = test_data[['PassengerId','name_categorie' ,'Pclass','Sex', 'Age','SibSp','Parch','Embarked', 'Ticket_categorie']]


#change_Pclass(train_set)
#change_Pclass(test_set)

change_SibSp(train_set)
change_SibSp(test_set)

change_Parch(train_set)
change_Parch(test_set)

change_Embarked(train_set)
change_Embarked(test_set)

#change_Fare(train_set)
#change_Fare(test_set)


# split null age

train_set_null = train_set[train_set['Age'].isna()]
test_set_null = test_set[test_set['Age'].isna()]

train_set = train_set[train_set['Age'].isna()==False]
test_set = test_set[test_set['Age'].isna()==False]


change_age_v3(train_set)
change_age_v3(test_set)


nb_var = train_set.shape[1]-1

train_set_temp, test_set_temp = train_test_split(train_set, test_size = 0.2, random_state=0 )

train_X_s = (train_set_temp.iloc[:,1:nb_var]).to_numpy()
train_Y_s = np.asanyarray(train_set_temp.iloc[:,nb_var])

test_X_s = (test_set_temp.iloc[:,1:nb_var]).to_numpy()
test_Y_s = np.asanyarray(test_set_temp.iloc[:,nb_var])
      
######################################################################################
train_set_null = train_set_null.drop(['Age'], axis=1)
test_set_null = test_set_null.drop(['Age'], axis=1)


nb_var_null = train_set_null.shape[1]-1

train_set_temp_null, test_set_temp_null = train_test_split(train_set_null, test_size = 0.2, random_state=0 )

train_X_s_null = (train_set_temp_null.iloc[:,1:nb_var_null]).to_numpy()
train_Y_s_null = np.asanyarray(train_set_temp_null.iloc[:,nb_var_null])

test_X_s_null = (test_set_temp_null.iloc[:,1:nb_var_null]).to_numpy()
test_Y_s_null = np.asanyarray(test_set_temp_null.iloc[:,nb_var_null])

      

In [65]:
model_reg_logistic = LogisticRegression(random_state =0, max_iter= 1000, class_weight="balanced" )

param = {'solver': ['newton-cg', 'lbfgs'], 'penalty': ["none"] } 

grid_reg_logistic = GridSearchCV(model_reg_logistic, param_grid = param, cv=4, scoring ='accuracy', n_jobs=-1 )
resultat_reg_logistic = grid_reg_logistic.fit(train_X_s, train_Y_s)
resultat_reg_logistic.best_params_

In [68]:
model_17_cw = LogisticRegression(random_state =0,penalty="none" , solver="newton-cg", max_iter= 1000, class_weight="balanced")

evaluation(model_17_cw, train_X_s, train_Y_s, test_X_s, test_Y_s, scoring="recall", plot=True )

In [13]:
model_DecisionTreeClassifier = DecisionTreeClassifier(random_state = 0 , class_weight = "balanced")
param = { 'criterion': ["gini", "entropy"], 'max_features':[2,3,4,5,6,7],
          }

grid_DecisionTreeClassifier = GridSearchCV(model_DecisionTreeClassifier, param_grid= param, cv = 4, scoring = "accuracy", n_jobs=-1)
resultat_grid_DecisionTreeClassifier = grid_DecisionTreeClassifier.fit(train_X_s, train_Y_s)
resultat_grid_DecisionTreeClassifier.best_params_

In [14]:
final_model_DecisionTreeClassifier = DecisionTreeClassifier(random_state = 0, class_weight = "balanced", criterion ='entropy', max_features = 5 )

evaluation(final_model_DecisionTreeClassifier, train_X_s, train_Y_s, test_X_s, test_Y_s, scoring="recall", plot=True)


In [15]:
#On conserve les noms de variable à part
liste_variables = list(train_set.columns[1:nb_var])
importances = final_model_DecisionTreeClassifier.feature_importances_
indices = np.argsort(importances)
# style du graphique 
plt.style.use('fivethirtyeight')
%matplotlib inline
plt.figure(1)
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [liste_variables[i] for i in indices])
plt.xlabel('Relative Importance')

In [19]:
model_RandomForestClassifier = RandomForestClassifier(random_state =0 )
param = {'criterion': ['gini','entropy'] ,'n_estimators' : [50, 100, 200, 1000], 'max_features':[2,3,4,5,6,7] }

grid_RandomForestClassifier = GridSearchCV(model_RandomForestClassifier, param, cv = 4, scoring = "accuracy", n_jobs=-1)
resultat_grid_RandomForestClassifier = grid_RandomForestClassifier.fit(train_X_s, train_Y_s)
resultat_grid_RandomForestClassifier.best_params_

In [18]:
final_model_RandomForestClassifier = RandomForestClassifier(random_state=0, criterion="entropy", n_estimators = 1000, max_features=2) 
y_pred = evaluation(final_model_RandomForestClassifier, train_X_s, train_Y_s, test_X_s, test_Y_s, scoring="recall", plot=True)

In [20]:
#On conserve les noms de variable à part
liste_variables = list(train_set.columns[1:nb_var])
importances = final_model_RandomForestClassifier.feature_importances_
indices = np.argsort(importances)
# style du graphique 
plt.style.use('fivethirtyeight')
%matplotlib inline
plt.figure(1)
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [liste_variables[i] for i in indices])
plt.xlabel('Relative Importance')

In [ ]:
model_AdaBoost = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(class_weight = "balanced"), random_state =0 )
param = {'base_estimator__criterion': ['gini','entropy'], 'base_estimator__min_samples_split' : [5, 10, 20, 40, 50,80, 100],
         'base_estimator__max_features':[2,3,4,5,6] , 'n_estimators' : [10, 50, 100],'learning_rate':[0.1,1] }

grid_AdaBoost = GridSearchCV(model_AdaBoost, param, cv = 4, scoring = "accuracy", n_jobs=-1)
resultat_grid_AdaBoost = grid_AdaBoost.fit(train_X_s, train_Y_s)
resultat_grid_AdaBoost.best_params_

In [ ]:
final_model_AdaBoost = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(class_weight="balanced", criterion="gini", max_features=2, min_samples_split=80),n_estimators = 10, learning_rate=0.1, random_state=0 ) 
evaluation(final_model_AdaBoost, train_X_s, train_Y_s, test_X_s, test_Y_s, scoring="recall" , plot =True)

In [ ]:

#############################################     MAX_DEPTH          ################################





In [21]:
model_RandomForestClassifier = RandomForestClassifier(class_weight="balanced", random_state =0 )
param = {'criterion': ['gini','entropy'] ,'n_estimators' : [1000, 500, 100], 'max_features':[2,3,4,5,6,7,8], 'max_depth' : list(range(2,10, 1)) }

grid_RandomForestClassifier = GridSearchCV(model_RandomForestClassifier, param, cv = 4, scoring = "accuracy", n_jobs=-1)
resultat_grid_RandomForestClassifier = grid_RandomForestClassifier.fit(train_X_s, train_Y_s)
resultat_grid_RandomForestClassifier.best_params_

In [22]:
final_model_RandomForestClassifier = RandomForestClassifier(random_state=0, criterion="gini", n_estimators = 100, max_features=4, max_depth=3) 
evaluation(final_model_RandomForestClassifier, train_X_s, train_Y_s, test_X_s, test_Y_s, scoring="recall", plot=True)

In [23]:
#On conserve les noms de variable à part
liste_variables = list(train_set.columns[1:nb_var])
importances = final_model_RandomForestClassifier.feature_importances_
indices = np.argsort(importances)
# style du graphique 
plt.style.use('fivethirtyeight')
%matplotlib inline
plt.figure(1)
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [liste_variables[i] for i in indices])
plt.xlabel('Relative Importance')

In [24]:
model_AdaBoost = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(class_weight = "balanced", random_state=0), random_state =0 )
param = {'base_estimator__criterion': ['gini','entropy'], 'base_estimator__max_depth' : list(range(2,10, 1)),
         'base_estimator__max_features':[2,3,4,5,6,7] , 'n_estimators' : [50, 100, 500],'learning_rate':[0.01,0.1,1]}

grid_AdaBoost = GridSearchCV(model_AdaBoost, param, cv = 4, scoring = "accuracy", n_jobs=-1)
resultat_grid_AdaBoost = grid_AdaBoost.fit(train_X_s, train_Y_s)
resultat_grid_AdaBoost.best_params_

In [27]:
final_model_AdaBoost = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(random_state=0, class_weight="balanced", criterion="entropy", max_features=2, max_depth=4),n_estimators = 50, learning_rate=0.01, random_state=0 ) 
evaluation(final_model_AdaBoost, train_X_s, train_Y_s, test_X_s, test_Y_s, scoring="recall" , plot =True)

In [ ]:
########################

In [26]:
model_Bagging = BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight = "balanced"), random_state =0 )
param = {'base_estimator__criterion': ['gini','entropy'], 'base_estimator__max_depth' : list(range(2,10, 1)), 'n_estimators' : [20,50,100],
         'bootstrap':[True], "bootstrap_features": [False]}

grid_Bagging = GridSearchCV(model_Bagging, param, cv = 4, scoring = "accuracy", n_jobs=-1)
resultat_grid_Bagging = grid_Bagging.fit(train_X_s, train_Y_s)
resultat_grid_Bagging.best_params_

In [28]:
final_model_BaggingClassifier = BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight = "balanced", criterion="gini", max_depth=4),bootstrap=True, bootstrap_features=False, n_estimators = 20, random_state=0) 
evaluation(final_model_BaggingClassifier, train_X_s, train_Y_s, test_X_s, test_Y_s, scoring="recall", plot=True)

In [ ]:
####################################################################################################################

In [50]:
model_DecisionTreeClassifier = DecisionTreeClassifier(random_state = 0 , class_weight = "balanced")
param = { 'criterion': ["gini", "entropy"], 'max_features':[2,3,4,5,6,7]}

grid_DecisionTreeClassifier = GridSearchCV(model_DecisionTreeClassifier, param_grid= param, cv = 4, scoring = "accuracy", n_jobs=-1)
resultat_grid_DecisionTreeClassifier = grid_DecisionTreeClassifier.fit(train_X_s_null, train_Y_s_null)
resultat_grid_DecisionTreeClassifier.best_params_

In [52]:
final_model_DecisionTreeClassifier = DecisionTreeClassifier(random_state = 0, class_weight = "balanced", criterion ='gini', max_features = 3 )

evaluation(final_model_DecisionTreeClassifier, train_X_s_null, train_Y_s_null, test_X_s_null, test_Y_s_null, scoring="recall", plot=True)


In [58]:
model_RandomForestClassifier = RandomForestClassifier(class_weight="balanced", random_state =0 )
param = {'criterion': ['gini','entropy'] ,'n_estimators' : [200, 100, 50], 'max_features':[2,3,4,5,6,7], 'max_depth' : list(range(2,6, 1)) }

grid_RandomForestClassifier = GridSearchCV(model_RandomForestClassifier, param, cv = 4, scoring = "accuracy", n_jobs=-1)
resultat_grid_RandomForestClassifier = grid_RandomForestClassifier.fit(train_X_s_null, train_Y_s_null)
resultat_grid_RandomForestClassifier.best_params_

In [59]:
final_model_RandomForestClassifier = RandomForestClassifier(random_state=0, criterion="gini", n_estimators = 200, max_features=3, max_depth=2) 
evaluation(final_model_RandomForestClassifier, train_X_s_null, train_Y_s_null, test_X_s_null, test_Y_s_null, scoring="recall", plot=True)

In [ ]:
###################################################################################################################

In [ ]:
data_pred = train_set.copy()
y_pred_train= final_model_RandomForestClassifier.predict((train_set.iloc[:,1:nb_var]).to_numpy())
data_pred['y_pred_train'] = y_pred_train
data_pred

In [ ]:
diff = data_pred[data_pred['Survived']!=data_pred['y_pred_train']]
diff

In [ ]:
diff['Sex'].value_counts(), diff['Pclass'].value_counts(), diff['Age'].value_counts(), diff['Survived'].value_counts()

In [ ]:
diff[diff['Sex']==1]

In [ ]:
diff[diff['Sex']==0]['Age'].value_counts()

In [ ]:
train_data[(train_data['Age']>16)&(train_data['Age']<20)&(train_data['Sex']==0)]['Survived'].value_counts()

In [ ]:
train_set['Age'].value_counts()

In [ ]:
diff.drop_duplicates(subset  = ['Age','SibSp','Parch'], keep = 'first')

In [ ]:
train_data[train_data.loc[:,'Name']=='Reuchlin, Jonkheer. John George']

In [ ]:
change_name(train_data)

In [ ]:
test_data['Name'].value_counts()

In [ ]:
############################# null (inplace =True) ##############################################################
estimator_null(train_set_null)
estimator_null(test_set_null)

test_set_null_temp = test_set_null[['PassengerId', 'y_pred']]
test_set_null_temp = test_set_null_temp.rename(columns={'y_pred':'Survived'})

In [ ]:
y_pred = final_model_RandomForestClassifier.predict((test_set.iloc[:,1:nb_var]).to_numpy())
y_pred

In [ ]:
#y_pred = final_model_AdaBoost.predict((test_set.iloc[:,1:nb_var]).to_numpy())
#y_pred

In [ ]:
output = pd.DataFrame({'PassengerId': test_set.PassengerId, 'Survived': y_pred})
output = pd.concat([output,test_set_null_temp], axis=0)

output = output.sort_values(by='PassengerId', ascending=True)
                       
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

In [ ]:
output

In [ ]:
train_set['Age'].value_counts(), train_set['Pclass'].value_counts(), train_set['SibSp'].value_counts(), train_set['Parch'].value_counts()

In [40]:
train_set_null

In [60]:
train_data_orig

# 